In [1]:
##### Data Prep, Training and Evaluation########
import cv2
import numpy as np

# the digits dataset
image = cv2.imread("D:/Computer_Vision_Recap/images/digits.png")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)
cv2.imshow("Digits Image", small)
cv2.waitKey(0)
cv2.destroyAllWindows()


# splits the image to 5000cells, each of 20 x 20 size giving a 4d array 
# i.e 50 x 100 x 20 x20
cells = [np.hsplit(row, 100) for row in np.vsplit(gray_image, 50)]

# converts the list data type to np array of shape(50, 100, 20, 20)
x = np.array(cells)
print("The shape of our cells array: " + str(x.shape))

# splits dset into training(0.7) and test(0.3) 
train = x[:,:70].reshape(-1, 400).astype(np.float32)
test = x[:,70:100].reshape(-1, 400).astype(np.float32)

# label creation for test and train data
# k = np.arange(10)
k = [0,1,2,3,4,5,6,7,8,9]
train_labels = np.repeat(k, 350)[:, np.newaxis]
test_labels = np.repeat(k, 150)[:, np.newaxis]

# inti and train knn
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret, result, neighbours, distance = knn.findNearest(test, k = 3)

# Accuracy check using test data
matches = result == test_labels
correct = np.count_nonzero(matches)     # gets total num of times matches val is not zero(true)
accuracy = correct * (100.0 / result.size)
print("Accuracy is: {0:.2f}%".format(accuracy))

The shape of our cells array: (50, 100, 20, 20)
Accuracy is: 93.47%


In [2]:
#### func to prepare an input image ####
import numpy as np
import cv2

def x_cord_contour(contour):
    # takes contours found and outputs the X centroid cords
    if(cv2.contourArea(contour) > 10):
        M = cv2.moments(contour)
        return (int(M["m10"] / M["m00"]))
    
def makeSquare(img_not_square):
    # takes xtracted contorus and makes it into a square image
    # and padds with black pixel when needed
    black = [0, 0, 0]
    img_dim = img_not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    
    if(height == width):    # true if img arg is in square dim
        square = img_not_square
        return square
    else:
        doublesize = cv2.resize(img_not_square, (2*width, 2*height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        if(height > width):
            pad = (height - width) / 2     # padding pixel
            pad = int(pad)
            print("Padding h > w: ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize, 0, 0, pad, \
                                                  pad, cv2.BORDER_CONSTANT, value = black)
        else:
            pad = (height - width) / 2     # padding pixel
            pad=int(pad)
            print("Padding h > w: ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize, pad, pad, 0,\
                                                  0, cv2.BORDER_CONSTANT, value = black)
            
    doublesize_square_dim = doublesize_square.shape
    return doublesize_square
            
def resize_to_pixel(dimensions, image):
    # resizes image to specified dimension needed for input i.e (20 x 20)
    # dimensionws is dim to resezie to and image is the squared_image
    buffer_pix = 4
    dimensions = dimensions - buffer_pix
    squared = image
    squared = np.array(squared)
    r = dimensions / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    black = [0, 0, 0]
    if(height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,
                                    cv2.BORDER_CONSTANT, value = black)
    if(height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,
                                    cv2.BORDER_CONSTANT, value = black)
    p = 2
    resizedImg = cv2.copyMakeBorder(resized, p,p,p,p,
                                   cv2.BORDER_CONSTANT, value=black)
    img_dim = resizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    print("Padded Height = ", height, "Width = ", width)
    return resizedImg
    

In [3]:
import cv2
import numpy as np

image = cv2.imread("D:/Computer_Vision_Recap/images/digits.jpg")
image = cv2.resize(image, (400,400), interpolation = cv2.INTER_AREA)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# blur image and canny detect op
blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)
edged = cv2.Canny(blurred, 30, 130)

# finds contours and sorts
contours, heirachy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# sorted_contours = sorted(contours, key = x_cord_contour, reverse = False)

# array to store entire number
full_number = []

for contour in contours:
    (x1, y1, x2, y2) = cv2.boundingRect(contour)      # gets boundgin box of the contour
    print("x2:{0}, y2{1}".format(x2, y2))
    
    if x2 >= 5 and y2 >= 25:          # contouring filtering condition
        roi = blurred[y1:y1+y2, x1:x1+x2]
        ret, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY_INV)
        squared_image = makeSquare(roi)                # gets the squared image
        final_image = resize_to_pixel(20, squared_image)     # gets the 20 x 20 image
        final_array = final_image.reshape((1, 400))     # i.e 1 x 400 i.e flattened 20 x 20
        final_array = final_array.astype(np.float32)
        # does the prediction
        ret, result, neighbours, dist = knn.findNearest(final_array, k = 3)
        num = str(int(float(result[0])))       # gets result of prediction
        full_number.append(num)
        
        # draw rect over image and show the prediction
        cv2.rectangle(image, (x1, y1), (x1+x2, y1+y2), (255, 0, 50), 2)
        cv2.putText(image, num, (x1, y1 + 155),
                   cv2.FONT_HERSHEY_PLAIN, 2, (0,0,200), 2)
        cv2.imshow("Image", image)
        cv2.waitKey(0)
        
cv2.destroyAllWindows()
print("The number deteted is: " + "".join(full_number))
        

x2:25, y249
Padding h > w:  24
Padded Height =  20 Width =  20
x2:30, y261
Padding h > w:  31
Padded Height =  20 Width =  20
x2:8, y28
x2:25, y216
x2:21, y260
Padding h > w:  39
Padded Height =  20 Width =  20
x2:86, y2338
Padding h > w:  252
Padded Height =  20 Width =  20
x2:61, y2340
Padding h > w:  279
Padded Height =  20 Width =  20
x2:37, y2276
Padding h > w:  239
Padded Height =  20 Width =  20
x2:128, y2337
Padding h > w:  209
Padded Height =  20 Width =  20
x2:400, y223
x2:43, y230
Padding h > w:  -13


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\core\src\copy.cpp:1421: error: (-215:Assertion failed) top >= 0 && bottom >= 0 && left >= 0 && right >= 0 && _src.dims() <= 2 in function 'cv::copyMakeBorder'


In [46]:
squared.shape

NameError: name 'squared' is not defined